In [1]:
# magics: ensures that any changes to the modules loaded below will be re-loaded automatically
%load_ext autoreload
%autoreload 2

# load packages
import numpy as np
import tools

# Exercise 5: Numerical Integration
Consider the numerical integration problem
$$
\int x^{2}dg(x),\,\,\,x\sim\mathcal{N}(0,1)
$$

Note that we can analytically show that for $f(x)=x^{2}$ (it is the
variance of $x$)
$$
\int f(x)g(x)dx=1
$$

In [15]:
# Define the function 
f = lambda x: x**2

### 1. Approximate the integral using *Monte Carlo integration*.

In [16]:
num_draws = 1000 # number of MC draws
np.random.seed(2023) # set seed to make sure the results are the same
x_mc = np.random.normal(size=num_draws) # draw from standard normal distribution
Efx_MC = np.mean(f(x_mc))
print(Efx_MC)

0.997344436151275


In [17]:
#Create a function for later use
def integrate_MC(f,num_points):
    np.random.seed(2023)
    x = np.random.normal(size=num_points) 
    return np.mean(f(x))

### 2. Approximate the integral using *Gauss-Hermite integration*.

Gauss-Hermite approximation:

\begin{align*}
    \int_{-\infty}^{\infty} f(x) \exp\{-x^2\} dx = \sum^n_{i=1} \omega_{gauss,i} f(x_{gauss,i}) +\text{error term}
\end{align*}

Hint: The goal is to rewrite the integration problem ,$\int x^{2}dg(x),\,\,\,x\sim\mathcal{N}(0,1)
$, with a change of variable such that $g(z) = \exp\{-x^2\}$, where z is the change of variable that makes the approximation work.

In [18]:
num_points = 5

# get "raw" hermite nodes and weights
x_gauss,w_gauss = tools.gauss_hermite(num_points)
x_gauss = x_gauss*np.sqrt(2) # scale nodes
w_gauss = w_gauss * (1/ np.sqrt(np.pi))

# evaluate expectation
Efx_gauss = f(x_gauss.T) @ w_gauss
print(Efx_gauss)

1.000000000000002


In [19]:
# construct function for use below
def integrate_gauss(f,num_points):                          # Define a function that takes a function f and a number of points as input
    x_gauss,w_gauss = tools.gauss_hermite(num_points)       # Get the nodes and weights
    return f(np.sqrt(2)*x_gauss.T)@w_gauss/np.sqrt(np.pi)   # Return the expectation

### 3. Compare the two methods across various number of grid points. How few grid points do you need for Gauss-Hermite integration?

In [20]:
num_array = [1,2,3,10,50,100,1000,3000,900000]

# We check the time
import time

for i,num in enumerate(num_array):   # i is the index, and num is the corresponding value: num_array[i]=num
    t0 = time.time()  # set the starting time
    print(f'Number of grid points:    {num}')
    print(f'MC:    {integrate_MC(f,num):.4f}')
    if num < 1500:
        print(f'gauss: {integrate_gauss(f,num):.4f}')
    print(f'True value:  {1:.4f}')
    t1 = time.time() # set the ending time
    print(f'time: {t1-t0:.8} seconds') # print the total time
    print(f'')


Number of grid points:    1
MC:    0.5065
gauss: 0.0000
True value:  1.0000
time: 0.00034618378 seconds

Number of grid points:    2
MC:    0.3059
gauss: 1.0000
True value:  1.0000
time: 0.00012397766 seconds

Number of grid points:    3
MC:    0.5385
gauss: 1.0000
True value:  1.0000
time: 0.00033593178 seconds

Number of grid points:    10
MC:    2.1859
gauss: 1.0000
True value:  1.0000
time: 0.00013899803 seconds

Number of grid points:    50
MC:    1.1788
gauss: 1.0000
True value:  1.0000
time: 0.00060892105 seconds

Number of grid points:    100
MC:    1.2652
gauss: 1.0000
True value:  1.0000
time: 0.0076768398 seconds

Number of grid points:    1000
MC:    0.9973
gauss: 1.0000
True value:  1.0000
time: 1.0791421 seconds

Number of grid points:    3000
MC:    0.9641
True value:  1.0000
time: 0.00035572052 seconds

Number of grid points:    900000
MC:    0.9997
True value:  1.0000
time: 0.02859807 seconds



### 4. Change the function f and see what happens.

In [21]:
num_array = [1,2,3,4,5,10,50,100,500,3000]

# New function
g = lambda x: np.exp(x)

for i,num in enumerate(num_array): # i is the index, and num is the corresponding value: num_array[i]=num
    print(f'Number of grid points:    {num}')
    if num < 1500:
        print(f'gauss: {integrate_gauss(g,num):.4f}')
    print(f'MC:    {integrate_MC(g,num):.4f}')
    print(f'True value:  {np.exp(1/2):.4f}')
    print(f'')

Number of grid points:    1
gauss: 1.0000
MC:    2.0374
True value:  1.6487

Number of grid points:    2
gauss: 1.5431
MC:    1.3801
True value:  1.6487

Number of grid points:    3
gauss: 1.6382
MC:    1.0425
True value:  1.6487

Number of grid points:    4
gauss: 1.6480
MC:    1.0985
True value:  1.6487

Number of grid points:    5
gauss: 1.6487
MC:    1.0594
True value:  1.6487

Number of grid points:    10
gauss: 1.6487
MC:    2.5204
True value:  1.6487

Number of grid points:    50
gauss: 1.6487
MC:    1.5695
True value:  1.6487

Number of grid points:    100
gauss: 1.6487
MC:    1.7393
True value:  1.6487

Number of grid points:    500
gauss: 1.6487
MC:    1.6985
True value:  1.6487

Number of grid points:    3000
MC:    1.5847
True value:  1.6487



As long as the polynomial we solve for is 2n-1, the Gauss version will be better.


Having a ton of shocks, Monte Carlo will be better, as the Gauss will compute the expectation for most things. Even though the chances for them happening are infintetisammly small 